# Hand Landmark モデルをEdgeTPU用にコンパイル

このノートブックは Google Colab で実行してください。

## 手順:
1. Google Colab (https://colab.research.google.com/) でこのノートブックを開く
2. 各セルを順番に実行
3. コンパイル完了後、モデルをダウンロード

## ステップ1: EdgeTPUコンパイラをインストール

In [ ]:
# EdgeTPUコンパイラのインストール
!curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | sudo apt-key add -
!echo "deb https://packages.cloud.google.com/apt coral-edgetpu-stable main" | sudo tee /etc/apt/sources.list.d/coral-edgetpu.list
!sudo apt-get update
!sudo apt-get install -y edgetpu-compiler

# バージョン確認
!edgetpu_compiler --version

## ステップ2: PINTO_model_zooからモデルをダウンロード

In [ ]:
# 作業ディレクトリ作成
!mkdir -p /content/hand_models
%cd /content/hand_models

# PINTO_model_zoo の Hand Detection モデルをダウンロード
!wget -q https://s3.ap-northeast-2.wasabisys.com/pinto-model-zoo/033_Hand_Detection_and_Tracking/resources.tar.gz

# 解凍
!tar -xzf resources.tar.gz

# Integer量子化モデルを探す
!find . -name "*hand_landmark*integer*" -o -name "*hand_landmark*quant*"

## ステップ3: モデルファイルの場所を確認

In [ ]:
# 解凍されたファイルを確認
!ls -lh

## ステップ4: 代替方法 - 直接tar.gzを展開

In [ ]:
# より詳細なリソースをダウンロード
!wget -q https://s3.ap-northeast-2.wasabisys.com/pinto-model-zoo/033_Hand_Detection_and_Tracking/resources.tar.gz -O resources_033.tar.gz
!tar -xzf resources_033.tar.gz

# 033ディレクトリ内を探索
!find 033_Hand_Detection_and_Tracking -name "*.tflite" | grep -i "hand_landmark.*integer"

## ステップ5: Integer量子化tar.gzを直接ダウンロード

In [ ]:
# Integer量子化リソースを直接ダウンロード（GitHubの生URL）
import urllib.request
import tarfile
import os

# resources_new.tar.gzをダウンロード（手動でアップロードする必要がある場合もあり）
print("⚠️ 注意: このステップは、前のステップでモデルが見つからなかった場合に実行してください")
print("\n以下のURLから手動でダウンロードして、Colabにアップロードすることもできます:")
print("https://github.com/PINTO0309/PINTO_model_zoo/tree/main/033_Hand_Detection_and_Tracking/03_integer_quantization")

## ステップ6: 手動アップロード方法

上記の自動ダウンロードが失敗した場合:

1. ローカルPCから `hand_landmark_new_256x256_integer_quant.tflite` をアップロード
2. Colabの左サイドバーのファイルアイコンをクリック
3. アップロードボタンでファイルをアップロード

In [ ]:
# ファイルアップロード用コード（実行するとファイル選択ダイアログが表示される）
from google.colab import files

print("ファイル選択ダイアログが表示されます。")
print("hand_landmark_new_256x256_integer_quant.tflite をアップロードしてください。")
uploaded = files.upload()

# アップロードされたファイル名を確認
for filename in uploaded.keys():
    print(f"✅ アップロード完了: {filename}")

## ステップ7: EdgeTPUコンパイラでコンパイル

In [ ]:
# モデルファイル名を指定（アップロードしたファイル名に合わせて変更）
model_filename = "hand_landmark_new_256x256_integer_quant.tflite"

# ファイルの存在確認
import os
if os.path.exists(model_filename):
    print(f"✅ モデルファイルが見つかりました: {model_filename}")
    print(f"ファイルサイズ: {os.path.getsize(model_filename) / (1024*1024):.2f} MB")
else:
    print(f"❌ エラー: {model_filename} が見つかりません")
    print("\n現在のディレクトリのファイル一覧:")
    !ls -lh

In [ ]:
# EdgeTPU用にコンパイル（所要時間: 5-30秒）
!edgetpu_compiler {model_filename}

# コンパイル結果を確認
print("\n=== コンパイル結果 ===")
!ls -lh *_edgetpu.tflite

## ステップ8: コンパイルログを確認

In [ ]:
# コンパイルログファイルを確認
import glob

log_files = glob.glob("*_edgetpu.log")
if log_files:
    print(f"コンパイルログ: {log_files[0]}")
    print("\n=== ログ内容 ===")
    !cat {log_files[0]}
else:
    print("ログファイルが見つかりません")

## ステップ9: EdgeTPUモデルをダウンロード

In [ ]:
# コンパイル済みモデルをダウンロード
from google.colab import files
import glob

# _edgetpu.tflite ファイルを探す
edgetpu_models = glob.glob("*_edgetpu.tflite")

if edgetpu_models:
    for model in edgetpu_models:
        print(f"📥 ダウンロード: {model}")
        files.download(model)
    print("\n✅ ダウンロード完了！")
    print("RaspberryPiの models/ ディレクトリにコピーしてください。")
else:
    print("❌ EdgeTPUモデルが見つかりません。コンパイルが失敗した可能性があります。")

## トラブルシューティング

### エラー: "Error opening file for reading"
- ファイルが正しくアップロードされていない
- ファイル名が間違っている
- 解決策: ステップ6の手動アップロードを実行

### エラー: "Model not quantized"
- モデルがInteger量子化されていない
- 解決策: 正しいInteger量子化版モデルを使用

### エラー: "Internal compiler error"
- モデルの構造がEdgeTPUに対応していない
- 解決策: PINTO_model_zooの別のバージョンを試す